In [1]:
import pickle
import re
import sys
from copy import deepcopy
from dataclasses import dataclass
from datetime import date, datetime, timedelta
from functools import partial
from importlib import reload
from pathlib import Path

import get_data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import pytorch_lightning as pl
import pytz
import talib
import wandb
import yfinance as yf
from datasets import assets
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from tools import inspect_code, plotting, training, wandb_api
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tqdm import tqdm
from wandb.keras import WandbCallback

import utils

log_wandb = False
repo_path = Path().resolve().parent
pio.renderers.default = "browser"

2022-05-16 23:24:32.647489: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-16 23:24:32.647531: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# if log_wandb:
#     import wandb

#     wandb_api.login()
#     run = wandb.init(
#         project="crypto-prediction",
#         group="Initial Gan",
#         job_type="test",
#     )


In [3]:
interesting_tickers = [
    "XRP",
    "EOS",
    "NEO",
    "ALGO",
    "SNX",
    "ETH",
    "AAVE",
    "BNB",
    "BTC",
    "DOT",
    "XTZ",
    "TRX",
    "ADA",
    "MATIC",
    "DOGE",
    "KLAY",
    "AVAX",
    "GRT",
    "SAND",
    "SOL",
    "MANA",
    "ATOM",
    "VET",
    "OMG",
]


In [4]:
def convert_to_timedelta(interval, ago):
    time_equivalence = {"d": "days", "h": "hours", "m": "minutes"}
    interval_value, interval_base = re.findall("\d+|\D+", interval)
    interval_value = ago * int(interval_value)
    interval_base = time_equivalence[interval_base]
    return timedelta(**{interval_base: interval_value})


def compute_metrics(predictions, targets):
    if isinstance(predictions, pd.DataFrame) or isinstance(predictions, pd.Series):
        predictions = predictions.to_numpy()
    if isinstance(targets, pd.DataFrame) or isinstance(targets, pd.Series):
        targets = targets.to_numpy()
    recall = recall_score(
        targets.reshape(-1, 1), predictions.reshape(-1, 1), zero_division=0
    )
    precision = precision_score(
        targets.reshape(-1, 1), predictions.reshape(-1, 1), zero_division=0
    )
    accuracy = accuracy_score(
        targets.reshape(-1, 1),
        predictions.reshape(-1, 1),
    )
    return (
        precision,
        recall,
        accuracy,
    )


In [5]:
reload(assets)
reload(get_data)
reload(utils)


def _concatenate_indicators(data, lag=5):
    data.loc[:, "useless"] = 0
    data.loc[:, "open"] = data.loc[:, "Open"].pct_change()
    data.loc[:, "high"] = data.loc[:, "High"].pct_change()
    data.loc[:, "low"] = data.loc[:, "Low"].pct_change()
    data.loc[:, "close"] = data.loc[:, "Close"].pct_change()
    data.loc[:, "volume"] = data.loc[:, "Volume"].pct_change()

    # for i in range(5):
    #     data.loc[:, f"open_{i}"] = data.loc[:, "Open"].shift(i).pct_change()
    #     data.loc[:, f"high_{i}"] = data.loc[:, "High"].shift(i).pct_change()
    #     data.loc[:, f"low_{i}"] = data.loc[:, "Low"].shift(i).pct_change()
    #     data.loc[:, f"close_{i}"] = data.loc[:, "Close"].shift(i).pct_change()
    #     data.loc[:, f"volume_{i}"] = data.loc[:, "Volume"].shift(i).pct_change()

    # data.loc[:, "high"] = data.loc[:, "High"].pct_change()
    # data.loc[:, "low"] = data.loc[:, "Low"].pct_change()
    # data.loc[:, "close"] = data.loc[:, "Close"].pct_change()
    # data.loc[:, "volume"] = data.loc[:, "Volume"].pct_change()

    # data.loc[:, "ATR_14"] = talib.ATR(
    #     data.loc[:, "High"], data.loc[:, "Low"], data.loc[:, "Open"], timeperiod=14
    # )
    # data.loc[:, "ATR_14"] = data.loc[:, "ATR_14"].pct_change()

    # data.loc[:, "ATR_70"] = talib.ATR(
    #     data.loc[:, "High"], data.loc[:, "Low"], data.loc[:, "Open"], timeperiod=70
    # )
    # data.loc[:, "ATR_70"] = data.loc[:, "ATR_70"].pct_change()

    # (
    #     data.loc[:, "macd"],
    #     data.loc[:, "macdsignal"],
    #     data.loc[:, "macdhist"],
    # ) = talib.MACD(data.loc[:, "Close"], fastperiod=12, slowperiod=26, signalperiod=9)
    # data.loc[:, "macd"] = data.loc[:, "macd"].pct_change()
    # data.loc[:, "macdsignal"] = data.loc[:, "macdsignal"].pct_change()
    # data.loc[:, "macdhist"] = data.loc[:, "macdhist"].pct_change()

    # (
    #     data.loc[:, "macd2"],
    #     data.loc[:, "macdsignal2"],
    #     data.loc[:, "macdhist2"],
    # ) = talib.MACD(data.loc[:, "Close"], fastperiod=26, slowperiod=52, signalperiod=9)
    # data.loc[:, "macd2"] = data.loc[:, "macd2"].pct_change()
    # data.loc[:, "macdsignal2"] = data.loc[:, "macdsignal2"].pct_change()
    # data.loc[:, "macdhist2"] = data.loc[:, "macdhist2"].pct_change()

    # data.loc[:, "ADX_14"] = talib.ADX(
    #     data.loc[:, "High"], data.loc[:, "Low"], data.loc[:, "Close"], timeperiod=14
    # )
    # data.loc[:, "ADX_14"] = data.loc[:, "ADX_14"].pct_change()

    # data.loc[:, "ADX_70"] = talib.ADX(
    #     data.loc[:, "High"], data.loc[:, "Low"], data.loc[:, "Close"], timeperiod=70
    # )
    # data.loc[:, "ADX_70"] = data.loc[:, "ADX_70"].pct_change()

    # m = list(range(1, 102, 5))
    # for mi in m:
    #     data.loc[:, f"ir_{mi}"] = data.loc[:, "Close"].shift(mi) / data.loc[:, "Open"].shift(mi) - 1
    # for mi in m:
    #     data.loc[:, f"cr_{mi}"] = (
    #         data.loc[:, "Close"].shift(1) / data.loc[:, "Close"].shift(mi + 1) - 1
    #     )
    # for mi in m:
    #     data.loc[:, f"or_{mi}"] = data.loc[:, "Open"] / data.loc[:, "Close"].shift(mi) - 1

    # scaler = MinMaxScaler()
    # l = (
    #     [
    #         "open",
    #         "high",
    #         "low",
    #         "close",
    #         "volume",
    #         "ATR_14",
    #         "ATR_70",
    #         "macd",
    #         "macdsignal",
    #         "macdhist",
    #         "macd2",
    #         "macdsignal2",
    #         "macdhist2",
    #         "ADX_14",
    #         "ADX_70",
    #     ]
    #     + [f"ir_{mi}" for mi in m]
    #     + [f"cr_{mi}" for mi in m]
    #     + [f"or_{mi}" for mi in m]
    # )
    # data.loc[
    #     :,
    #     l,
    # ] = scaler.fit_transform(data[l])
    data["Direction"] = (data["Close"].shift(-lag) > data["Open"]) & (data["Low"].shift(-1) > data["Open"])
    return data


class DataModule:
    def __init__(
        self,
        config,
        compute_metrics=None,
        inputs=None,
        save_klines=True,
    ):
        self.config = config
        self.compute_metrics = compute_metrics
        self.inputs = inputs
        self.save_klines = save_klines

    def setup(self):
        self.train_datapoints = []
        for input in self.inputs:
            dp = utils.create_asset(
                **input,
                interval=self.config["interval"],
                compute_metrics=self.compute_metrics,
                save_klines=self.save_klines,
            )
            if dp == []:
                continue
            dp.df = dp.df.dropna()
            dp.labels = dp.labels.dropna()

            common_index = dp.df.index.intersection(dp.labels.index)
            dp.df = dp.df.loc[common_index]
            dp.labels = dp.labels.loc[common_index].astype("bool")

            train_dp = assets.TrainAsset(
                ticker=input["ticker"],
                df=dp.df,
                labels=dp.labels,
                interval=self.config["interval"],
                compute_metrics=self.compute_metrics,
            )
            if not train_dp.isempty:
                self.train_datapoints.append(train_dp)

    def clean_datapoints(self, datapoints):
        return datapoints

    def concat_and_shuffle(self, features, labels):
        assert len(features) == len(labels)
        _features = np.concatenate(features, axis=0)
        _labels = np.concatenate(labels, axis=0)
        assert len(_features) == len(_labels)
        p = np.random.permutation(len(_features))
        return _features[p], _labels[p]

    def nest_train_test_val_split(
        self, datapoints, offset, train_size, val_size, test_size=0
    ):
        train_features = []
        train_labels = []
        val_features = []
        val_labels = []
        test_datapoints = []
        for dp in datapoints:
            train_features.append(dp.features[offset : train_size + offset])
            train_labels.append(dp.labels[offset : train_size + offset])
            val_features.append(
                dp.features[train_size + offset : train_size + val_size + offset]
            )
            val_labels.append(
                dp.labels[train_size + offset : train_size + val_size + offset]
            )

            test_datapoints.append(
                assets.TrainAsset(
                    ticker=dp.ticker,
                    df=dp.df.iloc[
                        train_size
                        + val_size
                        + offset : train_size
                        + val_size
                        + test_size
                        + offset
                    ],
                    labels=dp.labels.iloc[
                        train_size
                        + val_size
                        + offset : train_size
                        + val_size
                        + test_size
                        + offset
                    ],
                    interval=dp.interval,
                    compute_metrics=dp.compute_metrics,
                )
            )
        return (
            self.concat_and_shuffle(train_features, train_labels),
            self.concat_and_shuffle(val_features, val_labels),
            test_datapoints,
        )

    def _init_train_val_data(self, train_datapoints):
        train_datapoints = self.clean_datapoints(train_datapoints)
        if self.config["train_val_test_split"][0] > 1:
            train_size = int(self.config["train_val_test_split"][0])
        else:
            train_size = int(
                len(train_datapoints[0].df) * self.config["train_val_test_split"][0]
            )
        if self.config["train_val_test_split"][1] > 1:
            val_size = int(self.config["train_val_test_split"][1])
        else:
            val_size = int(
                len(train_datapoints[0].df) * self.config["train_val_test_split"][1]
            )
        if self.config["train_val_test_split"][2] > 1:
            test_size = int(self.config["train_val_test_split"][2])
        else:
            test_size = int(
                len(train_datapoints[0].df) * self.config["train_val_test_split"][2]
            )
        print(f"train_size: {train_size}, val_size: {val_size}, test_size: {test_size}")
        max_offset = len(train_datapoints[0].df) - (train_size + val_size + test_size)
        train_datasets = []
        val_datasets = []
        test_datapoints = []
        for offset in range(0, max_offset, val_size + test_size):
            train_dataset, val_dataset, test_datapoint = self.nest_train_test_val_split(
                train_datapoints, offset, train_size, val_size, test_size
            )
            train_datasets.append(train_dataset)
            val_datasets.append(val_dataset)
            test_datapoints.append(test_datapoint)
        return train_datasets, val_datasets, test_datapoints


config = {}

config["job_type"] = run.job_type if "run" in locals() else "test"
config["train_val_test_split"] = [300, 0, 100]
config["interval"] = "1h"
config["ago"] = 3000

inputs = [
    {
        "ticker": ticker,
        "beginning_date": datetime.combine(date.today(), datetime.min.time())
        - convert_to_timedelta(config["interval"], ago=config["ago"]),
        "ending_date": datetime.combine(date.today(), datetime.min.time()),
    }
    for ticker in interesting_tickers
]

dm = DataModule(
    config, partial(_concatenate_indicators, lag=1), inputs, save_klines=True
)
dm.setup()
train_datasets, val_datasets, test_datapoints = dm._init_train_val_data(
    dm.train_datapoints
)
print(f"Length training dataset: {len(train_datasets)}")
print(f"Length validation dataset: {len(train_datasets)}")
print(f"Length test dataset: {len(train_datasets)}")


train_size: 300, val_size: 0, test_size: 100
Length training dataset: 26
Length validation dataset: 26
Length test dataset: 26


In [6]:
config["train_val_test_split"] = [300, 0, 100]
for lag in [1]:
    dm = DataModule(
        config, partial(_concatenate_indicators, lag=lag), inputs, save_klines=True
    )
    dm.setup()
    train_datasets, val_datasets, test_datapoints = dm._init_train_val_data(
        dm.train_datapoints
    )

    avg_precision = 0
    avg_recall = 0
    avg_accuracy = 0
    counter = 0
    classifiers = []
    for idx, train_dataset in enumerate(tqdm(train_datasets)):
        try:
            val_dataset = val_datasets[idx]
            rf = SVC(C=10, class_weight="balanced")
            # rf = RandomForestClassifier(n_estimators=2000, max_depth=6)
            rf.fit(train_dataset[0], train_dataset[1])

            train_precision, train_recall, train_accuracy = compute_metrics(
                rf.predict(train_dataset[0]), train_dataset[1]
            )
            # precision, recall, accuracy = compute_metrics(
            #     rf.predict(val_dataset[0]), val_dataset[1]
            # )

            # avg_recall += recall
            # avg_accuracy += accuracy
            # avg_precision += precision
            # counter += 1

            # print("TRAINING")
            # print("Precision:", train_precision)
            # print("Recall:", train_recall)
            # print("Accuracy:", train_accuracy)
            # print("VALIDATION")
            # print("Precision:", precision)
            # print("Recall:", recall)
            # print("Accuracy:", accuracy)
            # print("----------------------------------------")
        except Exception as e:
            print(e)
        classifiers.append(rf)
    print(f"LAG: {lag}")
    print(f"precision: {avg_precision/counter}, recall: {avg_recall/counter}, accuracy: {avg_accuracy/counter},")

train_size: 300, val_size: 0, test_size: 100


100%|██████████| 26/26 [00:55<00:00,  2.15s/it]


LAG: 1


ZeroDivisionError: division by zero

In [48]:
for index, (classifier, test_datapoint) in enumerate(zip(classifiers, test_datapoints)):
    if index == 0:
        base_datapoints = {dp.ticker: deepcopy(dp) for dp in test_datapoint}
        for ticker in base_datapoints.keys():
            base_datapoints[ticker].predictions = classifier.predict(
                base_datapoints[ticker].features
            )
            # base_datapoints[ticker].probabilities = classifier.predict_proba(base_datapoints[ticker].features)[:, 1]
    else:
        for dp in test_datapoint:
            base_datapoints[dp.ticker].df = pd.concat(
                [base_datapoints[dp.ticker].df, dp.df]
            )
            base_datapoints[dp.ticker].labels = pd.concat(
                [base_datapoints[dp.ticker].labels, dp.labels]
            )
            base_datapoints[dp.ticker].predictions = np.concatenate(
                [
                    base_datapoints[dp.ticker].predictions,
                    classifier.predict(dp.features),
                ]
            )
            # base_datapoints[dp.ticker].probabilities = np.concatenate(
            #     [
            #         base_datapoints[dp.ticker].probabilities,
            #         classifier.predict_proba(dp.features)[:, 1],
            #     ]
            # )

base_precision = 0
base_recall = 0
base_accuracy = 0
for ticker, dp in base_datapoints.items():
    predictions = dp.predictions
    (
        base_datapoints[ticker].precision,
        base_datapoints[ticker].recall,
        base_datapoints[ticker].accuracy,
    ) = compute_metrics(predictions, dp.labels)

    base_precision += base_datapoints[ticker].precision
    base_recall += base_datapoints[ticker].recall
    base_accuracy += base_datapoints[ticker].accuracy

    print(
        f"{ticker}: {base_datapoints[ticker].precision} \t {base_datapoints[ticker].recall} \t {base_datapoints[ticker].accuracy}"
    )
print("AVERAGE")
print(base_precision/len(base_datapoints.values()), base_recall/len(base_datapoints.values()), base_accuracy/len(base_datapoints.values()))


XRP: 0.64366373902133 	 0.8221153846153846 	 0.8480769230769231
EOS: 0.632748538011696 	 0.8400621118012422 	 0.8396153846153847
NEO: 0.6323851203501094 	 0.8717948717948718 	 0.838076923076923
ALGO: 0.6027542372881356 	 0.8647416413373861 	 0.8215384615384616
SNX: 0.5929549902152642 	 0.9195751138088012 	 0.8196153846153846
ETH: 0.6717752234993615 	 0.8042813455657493 	 0.8519230769230769
AAVE: 0.5904761904761905 	 0.9029126213592233 	 0.828076923076923
BNB: 0.6856368563685636 	 0.7881619937694704 	 0.8584615384615385
BTC: 0.6832844574780058 	 0.7327044025157232 	 0.8515384615384616
DOT: 0.6028921023359288 	 0.9124579124579124 	 0.8426923076923077
XTZ: 0.6090621707060063 	 0.8810975609756098 	 0.8273076923076923
TRX: 0.6896551724137931 	 0.782608695652174 	 0.8488461538461538
ADA: 0.6223529411764706 	 0.9120689655172414 	 0.8569230769230769
MATIC: 0.6056955093099672 	 0.8694968553459119 	 0.8296153846153846
DOGE: 0.5865384615384616 	 0.8698752228163993 	 0.8396153846153847
KLAY: 0.559

In [69]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

m = 1

fig = make_subplots(
    rows=m,
    cols=1,
    subplot_titles=[dp.ticker for dp in base_datapoints.values()],
    horizontal_spacing=0.0001,
    vertical_spacing=0.1,
    shared_xaxes=True,
)

for index, (ticker, dp) in enumerate(base_datapoints.items()):
    if index >= m:
        break
    predictions = dp.predictions
    labels = dp.labels
    df = dp.df

    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df["Open"],
            high=df["High"],
            low=df["Low"],
            close=df["Close"],
        ),
        # go.Scatter(
        #     x=df.index,
        #     y=df["Open"],
        #     showlegend=False,
        #     line=dict(color="black", width=1),
        # ),
        row=index + 1,
        col=1,
    )

    lag = 2
    open = dp.df.loc[:, "Open"].iloc[predictions].to_frame(name="price")
    close = (
        dp.df.loc[:, "Close"]
        .shift(1)
        .iloc[[False] * lag + list(predictions[:-lag])]
        .to_frame(name="price")
    )

    if True in predictions[-lag:]:
        open = open.iloc[:-1]

    positive_mask = close["price"].to_numpy() > open["price"].to_numpy()
    positive_open = open.iloc[positive_mask]
    positive_open.loc[:, "index"] = list(range(0, len(positive_open)))
    positive_close = close.iloc[positive_mask]
    positive_close.loc[:, "index"] = np.array(list(range(0, len(positive_close)))) + 0.3
    positive_nan_df = pd.DataFrame(
        {"price": np.nan, "index": np.array(list(range(0, len(positive_open)))) + 0.5},
        index=positive_open.index,
    )
    positive_price_df = pd.concat(
        [positive_open, positive_close, positive_nan_df]
    ).sort_values("index")

    negative_mask = close["price"].to_numpy() < open["price"].to_numpy()
    negative_open = open.iloc[negative_mask]
    negative_open.loc[:, "index"] = list(range(0, len(negative_open)))
    negative_close = close.iloc[negative_mask]
    negative_close.loc[:, "index"] = np.array(list(range(0, len(negative_close)))) + 0.3
    negative_nan_df = pd.DataFrame(
        {"price": np.nan, "index": np.array(list(range(0, len(negative_open)))) + 0.5},
        index=negative_open.index,
    )
    negative_price_df = pd.concat(
        [negative_open, negative_close, negative_nan_df]
    ).sort_values("index")

    fig.add_trace(
        go.Scatter(
            x=positive_price_df.index,
            y=positive_price_df["price"],
            mode="lines",
            showlegend=False,
            line=dict(color="green"),
        ),
        row=index + 1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=negative_price_df.index,
            y=negative_price_df["price"],
            mode="lines",
            showlegend=False,
            line=dict(color="red"),
        ),
        row=index + 1,
        col=1,
    )
    
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]

fig.layout.on_change(zoom, 'xaxis.range')
fig.update_layout(height=450*m, width=1000, margin=dict(l=10, r=20, t=30, b=10))
fig.show()


/home/matias/.pyenv/versions/3.9.8/envs/binance/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [54]:
from datetime import datetime, timedelta
import vectorbt as vbt

vbt.settings.portfolio["fees"] = 0.001
vbt.settings.portfolio["slippage"] = 0.0025
data = vbt.Data.from_data(
    {dp.ticker: dp.df for dp in base_datapoints.values()},
    download_kwargs={},
)

# probabilities = vbt.Data.from_data(
#     {dp.ticker: dp.probabilities for dp in base_datapoints.values()},
#     download_kwargs={},
# )
# predictions = pd.DataFrame(
#     probabilities.get().values.argsort(axis=1) > 20,
#     columns=probabilities.get().columns,
#     index=data.get("Open").index,
# )
predictions = vbt.Data.from_data(
    {dp.ticker: (dp.predictions) & (dp.df["Low"].shift(-1) > dp.df["Open"]) for dp in base_datapoints.values()},
    download_kwargs={},
).get().set_index(data.get("Open").index)



In [55]:
# def apply_rf(*args, **kwargs):
#     proba = kwargs["proba"]
#     price = np.squeeze(np.stack(args[1:], axis=1))
#     length = price.shape[0]
#     try:
#         probabilities = rf.predict_proba(price)
#         direction = np.argmax(probabilities > proba, axis=1)
#     except ValueError:
#         direction = np.zeros(length)
#     return direction


# def plot_trix(trix, signal, column=None, fig=None):
#     fig = trix.vbt.plot(fig=fig)
#     fig = signal.vbt.plot(fig=fig)


# RF = vbt.IndicatorFactory(
#     input_names=list(data.data.values())[0].columns[6:],
#     output_names=["direction"],
#     # subplots=dict(
#     #     plot_outputs=dict(
#     #         plot_func=plot_trix,
#     #         resolve_trix=True,
#     #         resolve_signal=True,
#     #     )
#     # ),
# ).from_apply_func(
#     apply_rf,
#     proba=0.5,
# )
# direction = RF.run(
#     *data.get()[6:],
#     run_unique=True,
#     short_name="entries",
#     per_column=True,
#     pass_col=True
# )
# trend_ma = vbt.MA.run(data.get("Close"), window=50, ewm=True, run_unique=True)

ohlcstcx = vbt.OHLCSTCX.run(
    entries=predictions,
    open=data.get("Open"),
    high=data.get("High"),
    low=data.get("Low"),
    close=data.get("Close"),
    sl_stop=1,
    sl_trail=True,
    tp_stop=1,
)

exits = ~predictions | ohlcstcx.exits
entries =  ohlcstcx.entries


pf = vbt.Portfolio.from_signals(
    data.get("Open"),
    entries=entries,
    exits=exits,
    freq=timedelta(hours=1),
)
total_return = pf.total_return()
total_return.sort_values(ascending=False)
# total_return, total_return[total_return != 0].mean(), total_return[
#     total_return != 0
# ].median()

ohlcstcx_sl_stop  ohlcstcx_sl_trail  ohlcstcx_tp_stop  symbol
1                 True               1                 SNX       884.713758
                                                       AAVE      217.688148
                                                       GRT       208.297247
                                                       AVAX      206.120314
                                                       VET       114.720291
                                                       MANA       98.681459
                                                       SAND       98.457012
                                                       SOL        97.924417
                                                       OMG        96.581529
                                                       ATOM       62.458016
                                                       XTZ        56.012370
                                                       NEO        49.037215
                          

In [56]:
pf.loc[total_return.sort_values(ascending=False).index[-1]].stats()

Start                         2022-01-23 13:00:00+00:00
End                           2022-05-11 20:00:00+00:00
Period                                108 days 08:00:00
Start Value                                       100.0
End Value                                    358.076433
Total Return [%]                             258.076433
Benchmark Return [%]                         -17.962077
Max Gross Exposure [%]                            100.0
Total Fees Paid                              167.214081
Max Drawdown [%]                               4.077526
Max Drawdown Duration                  12 days 09:00:00
Total Trades                                        365
Total Closed Trades                                 365
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  49.863014
Best Trade [%]                                 7.533664
Worst Trade [%]                               -0

In [46]:
pf.loc[total_return.sort_values(ascending=False).index[1]].stats()

Start                         2022-01-23 13:00:00+00:00
End                           2022-05-11 20:00:00+00:00
Period                                108 days 08:00:00
Start Value                                       100.0
End Value                                  21868.814841
Total Return [%]                           21768.814841
Benchmark Return [%]                         -47.073791
Max Gross Exposure [%]                            100.0
Total Fees Paid                             2833.322626
Max Drawdown [%]                               0.811995
Max Drawdown Duration                   1 days 08:00:00
Total Trades                                        394
Total Closed Trades                                 394
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  85.532995
Best Trade [%]                                12.103071
Worst Trade [%]                               -0